In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests
from json import load
# import jsonschema
# from jsonschema import validate
import json

In [3]:
json_file_ghg ="global-greenhouse-gas-emissions.json"

In [ ]:
# {
# "title":"global-greenhouse-gas-emissions",
# "type":"object",
# "oneOf":[
# {
# "$ref":"#/definitions/global-greenhouse-gas-emissions"
# }
# ],
# "definitions":{
# "global-greenhouse-gas-emissions":{
# "properties":{
# "records":{
# "type":"array",
# "items":{
# "$ref":"#/definitions/global-greenhouse-gas-emissions_records"
# }
# }
# }
# },
# "global-greenhouse-gas-emissions_records":{
# "properties":{
# "fields":{
# "type":"object",
# "properties":{
# "year":{
# "type":"string",
# "format":"date",
# "title":"Year",
# "description":""
# },
# "iso_a3":{
# "type":"string",
# "title":"ISO_A3",
# "description":""
# },
# "world_region":{
# "type":"string",
# "title":"World Region",
# "description":""
# },
# "name":{
# "type":"string",
# "title":"Country",
# "description":""
# },
# "compound":{
# "type":"string",
# "title":"Compound",
# "description":""
# },
# "ipcc_description":{
# "type":"string",
# "title":"IPCC_description",
# "description":""
# },
# "ipcc_annex":{
# "type":"string",
# "title":"IPCC-Annex",
# "description":""
# },
# "ipcc":{
# "type":"string",
# "title":"IPCC",
# "description":""
# },
# "value":{
# "type":"number",
# "title":"Value (Gg)",
# "description":""
# }
# }
# }
# }
# }
# }
# }

In [4]:
gghg_file_df = pd.read_json(json_file_ghg)
gghg_file_df.head()

,datasetid,recordid,fields
0,global-greenhouse-gas-emissions,f54a80719f59eb970175add568e6ce7f33bfdd66,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
1,global-greenhouse-gas-emissions,706d949e60cc90591ca4f628818cf8d9b4f0c577,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
2,global-greenhouse-gas-emissions,939cf5d648c2b37258f0545860f9159de04babe3,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
3,global-greenhouse-gas-emissions,795477055ac695d8e8ddcd7f6dba993c1f3253bb,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
4,global-greenhouse-gas-emissions,a06fcfc69e16bcb8bbc892bc094f2cf12f54a77c,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."


In [5]:
gghg_fields_df = gghg_file_df.loc[:, "fields"]
gghg_fields_df.head()

0    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
1    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
2    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
3    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
4    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
Name: fields, dtype: object

In [7]:
norm_gghg_fields = pd.json_normalize(gghg_fields_df, max_level = 2)
norm_gghg_fields.head()

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
0,Annex_I,6D,Bulgaria,0.025106,1985,N2O,BGR,Central Europe,Other waste handling
1,Annex_I,6D,Bulgaria,0.025106,2011,N2O,BGR,Central Europe,Other waste handling
2,Annex_I,6D,Bulgaria,0.025106,1990,N2O,BGR,Central Europe,Other waste handling
3,Annex_I,6D,Bulgaria,0.025106,1992,N2O,BGR,Central Europe,Other waste handling
4,Annex_I,6D,Bulgaria,0.025106,1995,N2O,BGR,Central Europe,Other waste handling


In [30]:
name_country = norm_gghg_fields["name"].unique()
name_country

array(['Bulgaria', 'Bosnia and Herzegovina', 'Cyprus', 'Czech Republic',
       'Estonia', 'Croatia', 'Hungary', 'Lithuania', 'Latvia',
       'Macedonia, the former Yugoslav Republic of', 'Malta', 'Poland',
       'Romania', 'Armenia', 'Azerbaijan', 'Georgia',
       'Russian Federation', 'United Arab Emirates', 'Bahrain',
       'Iran, Islamic Republic of', 'Iraq', 'Israel', 'Jordan', 'Kuwait',
       'Lebanon', 'Oman', 'Qatar', 'Singapore', 'Thailand', 'Timor-Leste',
       'Viet Nam', 'Indonesia', 'Papua New Guinea', 'Japan',
       'American Samoa', 'Australia', 'Chad', 'Togo', 'Cook Islands',
       'Burundi', 'Fiji', 'Comoros', 'Micronesia, Federated States of',
       'Guam', 'Djibouti', 'Kiribati', 'Eritrea', 'New Caledonia',
       'Ethiopia', 'Kenya', 'Madagascar', 'Mauritius', 'Mayotte',
       'Reunion', 'Rwanda', 'Sudan', 'Somalia', 'Ecuador',
       'Falkland Islands (Malvinas)', 'French Guiana', 'Guyana', 'Peru',
       'Paraguay', 'Serbia and Montenegro', 'Suriname', '

In [47]:
# name_country = norm_gghg_fields["name"].unique()
# country_list = []
counrty_list = ['United States', 'Mexico', 'Brazil', 'Columbia', 'Venezuela','Chile','Peru','India','Bangladesh','Sri Lanka',
                'Myanmar', 'Pakistan', 'Russia', 'Ukraine', 'Kazakhstan', 'Azerbaijan', 'Armenia', 'Afghanistan', 'Uzbekistan',
                'South Africa', 'Nigeria', 'Ghana', 'Morocco', 'Egypt', 'Kenya', 'China', 'Japan', 'Hong Kong', 'Singapore', 
                'Thailand', 'Malaysia', 'Korea', 'Viet Nam', 'United Kingdom', 'France', 'Germany', 'Norway', 'Italy', 
                'Spain', 'Portugal', 'Sweden', 'Turkey', 'Greece', 'Saudi Arabia', 'Bahrain', 'Syrian Arab Republic', 
                'Iraq', 'Qatar', 'United Arab Emirates', 'Kuwait']

In [48]:
# check_dup_rows = clean_ssc_df.loc[clean_ssc_df["Mouse ID"] == dup_mice_list[0], :]
# check_new_df = norm_gghg_fields.loc[norm_gghg_fields["name"] == country_list[0], :]
# check_new_df= norm_gghg_fields.loc[:, "name" == "United States of America"]
norm_gghg_sort = norm_gghg_fields.sort_values("name")
# norm_gghg_sort.head()
norm_gghg_sort = norm_gghg_sort.dropna(how='any')
norm_gghg_sort

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
543677,Non-Annex_I,1.A.4,Afghanistan,2.828681e-02,1995,N2O,AFG,India +,Other Sectors
613179,Non-Annex_I,3D,Afghanistan,3.712901e-03,1974,N2O,AFG,India +,Solvent and other product use: other
613175,Non-Annex_I,1B2,Afghanistan,1.307460e-05,2001,N2O,AFG,India +,Fugitive emissions from oil and gas
613171,Non-Annex_I,1B2,Afghanistan,9.562420e-06,2002,N2O,AFG,India +,Fugitive emissions from oil and gas
613167,Non-Annex_I,1B2,Afghanistan,9.261660e-09,1971,N2O,AFG,India +,Fugitive emissions from oil and gas
...,...,...,...,...,...,...,...,...,...
352754,Non-Annex_I,1A4,Zimbabwe,1.275738e+03,2005,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Residential and other sectors
352758,Non-Annex_I,1A4,Zimbabwe,2.392590e+03,2008,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Residential and other sectors
352762,Non-Annex_I,1B1,Zimbabwe,6.324092e+02,1974,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Fugitive emissions from solid fuels
352842,Non-Annex_I,2A7,Zimbabwe,7.282750e-01,2012,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Production of other minerals


In [49]:
# mexico = norm_gghg_sort.loc[norm_gghg_sort['name'] == "Mexico"]
# mexico.head()

In [50]:
# df.loc[df['col1'].isin([value1, value2, value3, ...])]
# country1 = norm_gghg_sort.loc[norm_gghg_sort["name"].isin(["United States", "Mexico", "Brazil", "Columbia", "Venezuela"])]
country1 = norm_gghg_sort.loc[norm_gghg_sort["name"].isin(counrty_list)]
country1

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
543677,Non-Annex_I,1.A.4,Afghanistan,2.828681e-02,1995,N2O,AFG,India +,Other Sectors
613179,Non-Annex_I,3D,Afghanistan,3.712901e-03,1974,N2O,AFG,India +,Solvent and other product use: other
613175,Non-Annex_I,1B2,Afghanistan,1.307460e-05,2001,N2O,AFG,India +,Fugitive emissions from oil and gas
613171,Non-Annex_I,1B2,Afghanistan,9.562420e-06,2002,N2O,AFG,India +,Fugitive emissions from oil and gas
613167,Non-Annex_I,1B2,Afghanistan,9.261660e-09,1971,N2O,AFG,India +,Fugitive emissions from oil and gas
...,...,...,...,...,...,...,...,...,...
501749,Non-Annex_I,2.D,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501761,Non-Annex_I,2.D,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501741,Non-Annex_I,2.C,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Metal Industry
829745,Non-Annex_I,4.D,Viet Nam,4.649097e+02,2006,CH4,VNM,Southeastern Asia,Wastewater Treatment and Discharge
